In [3]:
with open('data/all.txt','r') as datafile:
    quotes = datafile.readlines()

In [67]:
list(sorted(set(quotes[0].split())))[:145]

['04',
 '05',
 '1',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '10000seat',
 '101',
 '102',
 '105',
 '11',
 '12',
 '125',
 '13',
 '132',
 '137',
 '14',
 '1400',
 '15',
 '16',
 '1600',
 '17',
 '17th',
 '18',
 '180000',
 '1832',
 '187',
 '18karat',
 '18th',
 '19',
 '19000',
 '1911',
 '1937',
 '1948',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1965',
 '1967',
 '1969',
 '1970',
 '1970s',
 '1980s',
 '1984',
 '1988',
 '1989',
 '19th',
 '19thcentury',
 '19year',
 '1st',
 '1yearold',
 '2',
 '20',
 '200',
 '2000',
 '2001',
 '2008',
 '200million',
 '2010',
 '2025',
 '20million',
 '20s',
 '20th',
 '20thcentury',
 '21',
 '215000',
 '21stcentury',
 '22',
 '22year',
 '24',
 '247',
 '25',
 '250000',
 '26',
 '28',
 '3',
 '30',
 '300',
 '30000',
 '30th',
 '31',
 '32',
 '33',
 '37',
 '3d',
 '3yearold',
 '4',
 '40',
 '400',
 '4000',
 '40000',
 '40hour',
 '40th',
 '42nd',
 '45',
 '48',
 '4th',
 '5',
 '50',
 '500',
 '5000',
 '50s',
 '50yearsold',
 '56',
 '58',
 '6',
 '60',
 '60000',
 '60s',
 '60yearold',
 

In [19]:
words = list(sorted(set(quotes[0].split())))[138:]

In [21]:
words[0] = '0' 
words[1] = 'a'

In [27]:
word_to_int = dict([(w,i) for i,w in enumerate(words)])
int_to_word = dict([(i,w) for i,w in enumerate(words)])

In [31]:
word_to_int, int_to_word

({'0': 0,
  'a': 1,
  'aback': 2,
  'abandon': 3,
  'abandoned': 4,
  'abandonment': 5,
  'abates': 6,
  'abba': 7,
  'abbreviating': 8,
  'abc': 9,
  'abdicate': 10,
  'abdicated': 11,
  'abductions': 12,
  'aberration': 13,
  'abiding': 14,
  'abilities': 15,
  'ability': 16,
  'abject': 17,
  'abjure': 18,
  'able': 19,
  'ably': 20,
  'abnormal': 21,
  'abnormality': 22,
  'abolish': 23,
  'abolished': 24,
  'abolition': 25,
  'abolitionists': 26,
  'abortion': 27,
  'abortions': 28,
  'about': 29,
  'above': 30,
  'abraham': 31,
  'abridgement': 32,
  'abroad': 33,
  'abrupt': 34,
  'absence': 35,
  'absences': 36,
  'absent': 37,
  'absolute': 38,
  'absolutely': 39,
  'absolutes': 40,
  'absorb': 41,
  'absorbed': 42,
  'abstract': 43,
  'abstractly': 44,
  'absurd': 45,
  'absurdist': 46,
  'absurdity': 47,
  'absurdly': 48,
  'abundance': 49,
  'abundant': 50,
  'abundantly': 51,
  'abuse': 52,
  'abuses': 53,
  'abusive': 54,
  'abyss': 55,
  'academia': 56,
  'academic': 57,

In [32]:
len(int_to_word)

13523

In [ ]:
with open('data/all.txt','r') as quotefile:
    quotes = quotefile.readlines()

In [49]:
with open('data/death.txt','r') as deathfile:
    deathquotes = deathfile.readlines()

In [211]:
import numpy as np
import sys
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback

In [34]:
t = Tokenizer()
t.fit_on_texts(quotes)
vocab_size = len(t.word_index) + 1

In [99]:
vocab_size, 
t.word_index, 

(13662,)

In [95]:
index_word = dict([(i,w) for (w,i) in t.word_index.items()])

In [97]:
np.save('index_word.npy',index_word)

In [82]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(deathquotes)
print(encoded_docs[0][:15])

[6, 77, 47, 560, 238, 12, 100, 5, 16, 741, 2, 113, 2, 13, 2663]


In [56]:
embeddings_index = np.load('data/embeddings_index.npy')

In [58]:
embeddings_index = embeddings_index.item()

In [59]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [69]:
embedding_matrix[9818]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [78]:
# np.save('word_index.npy',t.word_index)
# np.save('embedding_matrix.npy',embedding_matrix)

In [98]:
embedding_matrix.shape

(13662, 100)

## Generate Sequences, Next Sequences and Classes

## Death Doc

In [110]:
maxlen = 100
step = 1
seq_death = []
next_seq_death = []

In [111]:
death_doc = encoded_docs[0]
quote_len_death = len(death_doc)

In [112]:
for i in range(0, quote_len_death - maxlen, step):
    seq_death.append(death_doc[i: i + maxlen])
    next_seq_death.append(death_doc[i + maxlen])

print('nb sequences:', len(seq_death))

seq_death = np.asarray(seq_death)
next_seq_death = np.asarray(next_seq_death)

nb sequences: 23240


In [114]:
labels_death = np.zeros(next_seq_death.shape)

In [117]:
labels_death

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

## Funny Doc

In [118]:
with open('data/funny.txt','r') as funnyfile:
    funnyquotes = funnyfile.readlines()

In [119]:
encoded_docs = t.texts_to_sequences(funnyquotes)
funny_doc = encoded_docs[0]

In [123]:
maxlen = 100
step = 1
seq_funny = []
next_seq_funny = []

quote_len_funny = len(funny_doc)

In [124]:
for i in range(0, quote_len_funny - maxlen, step):
    seq_funny.append(funny_doc[i: i + maxlen])
    next_seq_funny.append(funny_doc[i + maxlen])

print('nb sequences:', len(seq_funny))

seq_funny = np.asarray(seq_funny)
next_seq_funny = np.asarray(next_seq_funny)

nb sequences: 24396


In [125]:
labels_funny = np.ones(next_seq_funny.shape)

In [126]:
labels_funny

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

## Combine

In [130]:
X = np.vstack([seq_death,seq_funny])

In [132]:
X.shape

(47636, 100)

In [135]:
labels_death = np.reshape(labels_death,(labels_death.shape[0],1))
labels_funny  = np.reshape(labels_funny,(labels_funny.shape[0],1))

In [136]:
y = np.vstack([labels_death,labels_funny])

In [137]:
y.shape

(47636, 1)

In [139]:
from sklearn.model_selection import train_test_split

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

# Sequence Classification Keras

In [157]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)

In [158]:
model = Sequential()
model.add(e)

In [159]:
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

In [160]:
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          1366200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,429,133
Trainable params: 62,933
Non-trainable params: 1,366,200
_________________________________________________________________
None


In [164]:
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=24)
'''Train on 38108 samples, validate on 9528 samples
Epoch 1/10
38108/38108 [==============================] - 80s 2ms/step - 
loss: 0.0391 - acc: 0.9859 - val_loss: 4.4262e-04 - val_acc: 0.9999'''

'Train on 38108 samples, validate on 9528 samples\nEpoch 1/10\n38108/38108 [==============================] - 80s 2ms/step - \nloss: 0.0391 - acc: 0.9859 - val_loss: 4.4262e-04 - val_acc: 0.9999'

# Text Generation using Word Embeddings

In [165]:
seq_funny, next_seq_funny

(array([[    6,   343,    77, ...,    57,  1419,    40],
        [  343,    77,     5, ...,  1419,    40,    42],
        [   77,     5,    41, ...,    40,    42,  1038],
        ..., 
        [   73,    13, 10360, ...,  3394,  1798,    23],
        [   13, 10360,     8, ...,  1798,    23,     1],
        [10360,     8,     1, ...,    23,     1,    53]]),
 array([  42, 1038,   75, ...,    1,   53, 1239]))

In [166]:
from keras.utils import to_categorical

In [167]:
y = to_categorical(next_seq_funny, num_classes=vocab_size)

In [168]:
X = seq_funny

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [214]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)

In [215]:
model = Sequential()
model.add(e)

In [216]:
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

In [217]:
model.add(LSTM(100))
model.add(Dense(y.shape[1]))

In [218]:
model.add(Activation('softmax'))
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 100)          1366200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 32)           9632      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_6 (Dense)              (None, 13662)             1379862   
_________________________________________________________________
activation_3 (Activation)    (None, 13662)             0         
Total params: 2,808,894
Trainable params: 1,442,694
Non-trainable params: 1,366,200
__________________________________________________________

In [219]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [220]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024,callbacks=[print_callback])

Train on 19516 samples, validate on 4880 samples
Epoch 1/10
19516/19516 [==============================] - 62s 3ms/step - loss: 8.8853 - acc: 0.0223 - val_loss: 7.5319 - val_acc: 0.0398

----- Generating text after Epoch: 0
----- diversity: 0.2
[537, 10202, 26, 15, 4964, 3674, 8, 201, 14, 3658, 3149, 4, 7, 57, 178, 151, 1946, 21, 34, 44, 126, 9, 18, 734, 1946, 3658, 3149, 5, 56, 5859, 598, 4, 245, 5, 79, 53, 7, 59, 32, 1306, 57, 59, 3, 335, 22, 53, 8, 201, 14, 42, 59, 42, 154, 44, 325, 7, 886, 195, 6, 131, 50, 1, 6661, 280, 65, 258, 53, 18, 10203, 44, 4823, 78, 4, 11, 5, 172, 3, 46, 10204, 262, 7, 58, 27, 6662, 6663, 763, 258, 81, 53, 658, 7, 124, 2951, 812, 5, 53, 21, 32, 103, 27]
----- Generating with seed: ['book', 'carrie', 'was', 'my', 'alter', 'ego', 'in', 'real', 'life', 'sarah', 'jessica', 'and', 'i', 'dont', 'look', 'anything', 'alike', 'but', 'people', 'do', 'say', 'that', 'we', 'sound', 'alike', 'sarah', 'jessica', 'is', 'an', 'adorable', 'girl', 'and', 'she', 'is', 'very', 

i i i i i i i i i i i i i i i i i i i i 
Epoch 7/10
12288/19516 [=================>............] - ETA: 21s - loss: 6.0783 - acc: 0.0422

KeyboardInterrupt: 

In [208]:
index_word = np.load('index_word.npy')
index_word = index_word.item()

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    # preds = preds[1:]
    # preds = np.asarray(preds).astype('float64')
    # preds = np.log(preds) / temperature
    # exp_preds = np.exp(preds)
    # preds = exp_preds / np.sum(exp_preds)
    # probas = np.random.multinomial(1, preds, 1)
    return np.argmax(preds)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(funny_doc) - maxlen - 1)
    for diversity in [0.2]: # 0.5, 1.0, 1.2
        print('----- diversity:', diversity)

        generated = ''
        sentence = funny_doc[start_index: start_index + maxlen]
        print(sentence)
        generated.join([str([index_word[value]]).join(' ') for value in sentence])
        print('----- Generating with seed: %s'%[index_word[word] for word in sentence])
        #sys.stdout.write(generated)

        for i in range(20):
            x_pred = np.reshape(sentence,(1, maxlen))

            preds = model.predict(x_pred, verbose=0)
            preds = preds[0]
            # print(preds.shape)
            next_index = sample(preds, diversity)
            #print(next_index)
            next_char = index_word[next_index]

            generated.join(str(next_char))
            sentence = np.append(sentence[1:],next_index)

            sys.stdout.write(next_char)
            sys.stdout.write(" ")
            sys.stdout.flush()
        print()